In [7]:
# Import Necessary Libraries
import pandas as pd
import os
import pyarrow 
from azure.storage.blob import BlobServiceClient,BlobClient 
from dotenv import load_dotenv

load_dotenv()

True

In [8]:
print(os.getenv('deploymentname'))

mordecaisipcostorage_1754453741447


In [9]:
file_basedir = './data'

zikodf = pd.read_csv(os.path.join(file_basedir,'raw','ziko.csv'))

In [10]:
zikodf.columns

Index(['Transaction_ID', 'Date', 'Customer_ID', 'Product_ID', 'Quantity',
       'Unit_Price', 'Total_Cost', 'Discount_Rate', 'Sales_Channel',
       'Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status',
       'Customer_Satisfaction', 'Item_Returned', 'Return_Reason',
       'Payment_Type', 'Taxable', 'Region', 'Country', 'Customer_Name',
       'Customer_Phone', 'Customer_Email', 'Customer_Address',
       'Product_List_Title'],
      dtype='object')

In [11]:
# DAta cleaning and transformation

zikodf.fillna({
    'Unit_Price': zikodf['Unit_Price'].mean(),
    'Total_Cost':zikodf['Total_Cost'].mean(),
    'Discount_Rate': 0.0,
    'Return_Reason': 'Unknown'
},inplace=True)



In [12]:
zikodf['Date'] = pd.to_datetime(zikodf['Date'])

In [13]:
#customer table 
customer = zikodf[['Customer_ID', 'Customer_Name','Customer_Phone','Customer_Email','Customer_Address']].copy().drop_duplicates().reset_index(drop=True)


In [14]:
# product table 
product = zikodf[['Product_ID','Product_List_Title','Quantity','Unit_Price','Total_Cost','Discount_Rate']].copy().drop_duplicates().reset_index(drop=True)

product.head()

,Product_ID,Product_List_Title,Quantity,Unit_Price,Total_Cost,Discount_Rate
0,536,Product 53,3,120.436821,8265.374549,0.20
1,523,Product 33,6,475.724994,4047.850479,0.00
2,535,Product 6,3,146.400556,5096.553818,0.05
3,546,Product 68,6,19.373194,8194.281993,0.00
4,556,Product 89,8,193.221313,8331.329249,0.20


In [15]:
customer.columns

Index(['Customer_ID', 'Customer_Name', 'Customer_Phone', 'Customer_Email',
       'Customer_Address'],
      dtype='object')

In [16]:
# Transactioni_fact_table 

transaction_fact = zikodf.merge(customer, on=['Customer_ID', 'Customer_Name','Customer_Phone','Customer_Email','Customer_Address'],  how='left') \
                    .merge(product,on=['Product_ID','Product_List_Title','Quantity','Unit_Price','Total_Cost','Discount_Rate'], how='left') \
                    [['Transaction_ID','Date','Customer_ID','Product_ID','Total_Cost', 'Sales_Channel', 'Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status', 'Customer_Satisfaction', 'Item_Returned', 'Return_Reason', 'Payment_Type', 'Taxable', 'Region', 'Country']]

transaction_fact


                    

,Transaction_ID,Date,Customer_ID,Product_ID,Total_Cost,Sales_Channel,Order_Priority,Warehouse_Code,Ship_Mode,Delivery_Status,Customer_Satisfaction,Item_Returned,Return_Reason,Payment_Type,Taxable,Region,Country
0,200,2020-01-01 20:32:26,1086,536,8265.374549,Online,High,WH-3,2-Day,Cancelled,Neutral,False,Wrong Item,Wire Transfer,False,West,Canada
1,321,2020-01-02 06:55:08,1078,523,4047.850479,Reseller,Critical,WH-1,Overnight,Backorder,Satisfied,True,Damaged,PayPal,True,South,Mexico
2,989,2020-01-06 08:12:58,1077,535,5096.553818,Direct,Critical,WH-1,Overnight,Pending,Unsatisfied,True,Damaged,PayPal,True,West,Canada
3,682,2020-01-07 22:03:15,1027,546,8194.281993,Reseller,Medium,WH-1,Express,Pending,Unsatisfied,False,Wrong Item,Cash,True,South,Canada
4,484,2020-01-07 07:08:06,1052,556,8331.329249,Direct,Low,WH-2,2-Day,Delivered,Satisfied,True,Late,Cash,False,South,Mexico
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000,250,2020-01-05 02:22:42,1095,533,1138.724711,Reseller,Medium,WH-3,Express,Delivered,Unsatisfied,True,Damaged,Credit Card,True,South,Canada
1001,412,2020-01-07 11:27:34,1008,542,1786.865600,Online,Medium,WH-3,2-Day,Cancelled,Neutral,True,Not Specified,Cash,True,South,USA
1002,816,2020-01-06 02:48:39,1060,511,6443.682500,Partner,Medium,WH-3,2-Day,Cancelled,Unsatisfied,True,Not Specified,Cash,False,West,Mexico
1003,702,2020-01-09 08:38:55,1054,504,4814.744796,Online,Medium,WH-2,Regular,Cancelled,Satisfied,False,Damaged,Cash,True,West,Canada


In [17]:
customer.to_csv(os.path.join(file_basedir,'processed','customer.csv'),index=False)
product.to_csv(os.path.join(file_basedir,'processed','product.csv'),index=False)
transaction_fact.to_csv(os.path.join(file_basedir,'processed','transaction.csv'),index=False)

In [18]:
# data loading
# laod the environment variables from the .env files

azure_storage_name = os.getenv('azure_storage_name')
connection_string = os.getenv('azure_connection_string')
container_name = os.getenv('container_name')

In [21]:
# create a blobserviceclient object 
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_client = blob_service_client.get_container_client(container_name)

# load data to azure blog storage
files = [
    (zikodf, 'data/raw/zipco_customer.csv'),
    (product, 'data/processed/zipco_product.csv'),
    (customer, 'data/processed/zipco_customer.csv'),
    (transaction_fact, 'data/processed/zipco_transaction.csv'),
]

for file,blob_name in files:
    blob_client = container_client.get_blob_client(blob_name)
    output = file.to_csv(index=False)
    blob_client.upload_blob(output, overwrite=True)
    print(f'{blob_name} loaded into azure blob storage')


data/raw/zipco_customer.csv loaded into azure blob storage
data/processed/zipco_product.csv loaded into azure blob storage
data/processed/zipco_customer.csv loaded into azure blob storage
data/processed/zipco_transaction.csv loaded into azure blob storage
